In [1]:
# the main loop in the fine-tuning process
import os
from datasets import load_dataset
from omegaconf import DictConfig, OmegaConf
from ft_src.sft_trainer import CustomSFTTrainer
from ft_src.model import Model
import torch

cfg = OmegaConf.load("cfg/qwen2_5-vl_train.yaml")
# 1. load the model
vlm_model = Model(cfg['model'])

# 2. load the data
dataset = load_dataset(cfg.dataset.dataset_id, split='train')

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


Model parameters: {'pretrained_model_name_or_path': 'Qwen/Qwen2.5-VL-7B-Instruct', 'torch_dtype': torch.float16, 'device_map': 'auto'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


In [2]:
from ft_src.sft_dataset import generate_description
data_sample = dataset[100]
original_hddl = generate_description(data_sample, vlm_model.model, vlm_model.processor)

# load the lora model
vlm_model.model.load_adapter(cfg.trainer.output_dir)
ft_hddl = generate_description(data_sample, vlm_model.model, vlm_model.processor)

In [3]:
import pandas as pd
from IPython.display import display, HTML

def compare_generations(base_gen, ft_gen):
    # Ensure strings are displayed with line breaks
    base_gen = base_gen.encode().decode('unicode_escape')
    ft_gen = ft_gen.encode().decode('unicode_escape')

    # Create a DataFrame
    df = pd.DataFrame({
        'Base Generation': [base_gen],
        'Fine-tuned Generation': [ft_gen]
    })

    # Style the DataFrame for multiline rendering and fixed-width formatting
    styled_df = df.style.set_table_styles([
        {
            'selector': 'td',
            'props': [
                ('text-align', 'left'),
                ('white-space', 'pre-wrap'),
                ('font-family', '"Courier New", monospace'),
                ('border', '1px solid black'),
                ('padding', '10px'),
                ('vertical-align', 'top'),
                ('width', '500px'),  # adjust width as needed
                ('overflow-wrap', 'break-word')
            ]
        },
        {
            'selector': 'th',
            'props': [
                ('text-align', 'left'),
                ('font-family', '"Courier New", monospace'),
                ('border', '1px solid black'),
                ('padding', '10px')
            ]
        }
    ])

    # Display in notebook or IPython
    display(HTML(styled_df.to_html()))

compare_generations(original_hddl[0], ft_hddl[0])


,Base Generation,Fine-tuned Generation
0,```hddl (object banana wp1s) (object apple wp2s) (object scissors wp3s) (object tape wp4s) (object basket wp5s) (goal (at banana wp5f)) (goal (at apple wp5f)) (move_object banana wp5f) (move_object apple wp5f) (move_object scissors wp1f) (move_object tape wp1f) ```,(hddl (objects (at food1 banana wp1s)) (at food2 apple wp2s) (at container1 basket wp1f) (at other1 scissors wp2f) (at other2 tape wp3f) ) (only_permissible_actions move_object move_to_container) ;(move_object food1 container1) ;(move_to_container food1 container1) ;(move_object food2 container1) ;(move_to_container food2 container1) ;(move_object other1 wp1f) ;(move_object other2 wp1f) )
